In [27]:
# Librerias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

In [38]:
# Parametros de seguridad
umbral_pico = 0.5

max_i_r_ple1 = 17
max_i_s_ple1 = 18
max_i_t_ple1 = 17

max_i_r_ple7 = 13
max_i_s_ple7 = 28
max_i_t_ple7 = 32

tiempo_min = pd.Timedelta(minutes=5)
cant_min_picos = 10
tiempo_min_picos = pd.Timedelta(minutes=60)



In [29]:
# Diccionario de parametros de seguridad

limites_corriente = {
    'PLE1': {
        'R': max_i_r_ple1,
        'S': max_i_s_ple1,
        'T': max_i_t_ple1,
    },
    'PLE7': {
        'R': max_i_r_ple7,
        'S': max_i_s_ple7,
        'T': max_i_t_ple7,
    }
}

In [30]:
# Abrir los graficos en el navegador

pio.renderers.default = 'browser'

In [31]:
# Funcion para cargar datos

from glob import glob
import pandas as pd

def cargar_maquina(base_path, maquina):
    paths = glob(f"{base_path}/{maquina}/**/*.csv", recursive=True)

    dfs = []
    for path in paths:
        df = pd.read_csv(path)
        df['maquina'] = maquina
        dfs.append(df)

    return (
        pd.concat(dfs, ignore_index=True)
          .sort_values('temporal_placa')
          .reset_index(drop=True)
    )


In [32]:
# Carga de datos

df_ple1 = cargar_maquina("../data", "PLE1")
df_ple7 = cargar_maquina("../data", "PLE7")

In [33]:
def preparar_df(df):
    df = df.copy()

    # Timestamp
    df['temporal_placa'] = pd.to_datetime(df['temporal_placa'])
    df['hora'] = df['temporal_placa'].dt.hour
    df['minuto'] = df['temporal_placa'].dt.minute

    # Turnos
    def asignar_turno(hora, minuto):
        t = hora * 60 + minuto

        # Pausas
        if 12*60 <= t < 12*60 + 30:
            return 'ALMUERZO'
        if 22*60 <= t < 22*60 + 30:
            return 'CENA'

        # Turnos
        if 5*60 <= t < 17*60:
            return 'TURNO MAÑANA'
        if 17*60 <= t < 22*60:
            return 'TURNO TARDE'
        if (t >= 22*60 + 30) or (t < 1*60):
            return 'TURNO TARDE'

        return 'FUERA_TURNO'

    df['turno'] = df.apply(
        lambda x: asignar_turno(x['hora'], x['minuto']),
        axis=1
    )
        
    # Potencias totales por timestamp
    df['p_activa_total'] = (
        df['potencia_a_r'] +
        df['potencia_a_s'] +
        df['potencia_a_t']
    )

    df['q_reactiva_total'] = (
        df['potencia_r_r'] +
        df['potencia_r_s'] +
        df['potencia_r_t']
    )

    return df


In [34]:

# Ajuste de datos de las plegadoras

df_ple1 = preparar_df(df_ple1)
df_ple7 = preparar_df(df_ple7)

df_all = pd.concat([df_ple1, df_ple7], ignore_index=True)

In [40]:
# Funcion detectar_picos

def detectar_picos(signal, timestamp, umbral_pico):
    dt = timestamp.diff().dt.total_seconds()
    valid = dt > 0

    dI_dt = signal.diff() / dt

    return (
        valid &
        dI_dt.notna() &
        (dI_dt > umbral_pico)
    )

In [41]:
# Calculo la media de las corrientes excluyendo los picos

def media_sin_picos(s, q = 0.9):
    return s[s <= s.quantile(q)].mean()

medias_por_fase = (
    df_all
    .groupby('maquina')
    .agg(
        i_r_media=('corriente_r', media_sin_picos),
        i_s_media=('corriente_s', media_sin_picos),
        i_t_media=('corriente_t', media_sin_picos),
    )
)

medias_por_fase

,i_r_media,i_s_media,i_t_media
maquina,,,
PLE1,16.187595,17.210818,16.276914
PLE7,11.065071,23.275452,26.435411


In [53]:
# Filtrado de picos + timestamp

peaks_rows = []

for maq, g in df_all.groupby('maquina'):

    g = g.sort_values('temporal_placa').reset_index(drop=True)

    for fase, col in {'r': 'corriente_r',
                      's': 'corriente_s',
                      't': 'corriente_t'}.items():

        media_ref = medias_por_fase.loc[maq, f'i_{fase}_media']

        # 1) detectar flancos por derivada
        mask_picos = detectar_picos(
            g[col],
            g['temporal_placa'],
            umbral_pico
        )

        # 2) filtrar por amplitud: 3 × media base
        mask_grande = g[col] >= 2.4 * media_ref

        # 3) pico válido = derivada fuerte + amplitud grande
        mask_final = mask_picos & mask_grande

        p = g[mask_final]

        for _, r in p.iterrows():
            peaks_rows.append({
                'maquina': maq,
                'fase': fase,
                'temporal_placa': r['temporal_placa'],
                'valor_corriente': r[col],
                'media_base': media_ref
            })

df_peaks_all = pd.DataFrame(peaks_rows)


In [55]:
# Alarmas

alarmas = []

# A) Sobrecorriente sostenida por fase

for maq, g in df_all.groupby('maquina'):

    g = g.sort_values('temporal_placa')

    if maq not in limites_corriente:
        continue

    for fase, col in {'R':'corriente_r','S':'corriente_s','T':'corriente_t'}.items():

        limite = limites_corriente[maq][fase]

        g_fase = g[['temporal_placa', col]].copy()
        g_fase['exceso'] = g_fase[col] > limite
        g_fase['bloque'] = (g_fase['exceso'] != g_fase['exceso'].shift()).cumsum()

        for _, b in g_fase[g_fase['exceso']].groupby('bloque'):

            duracion = b['temporal_placa'].iloc[-1] - b['temporal_placa'].iloc[0]

            if duracion >= tiempo_min:
                alarmas.append({
                    'maquina': maq,
                    'tipo_alarma': 'SOBRECORRIENTE',
                    'descripcion': 'Sobrecorriente sostenida por encima del límite',
                    'fase': fase,
                    'valor_medido': b[col].mean(),
                    'valor_limite': limite,
                    'fecha_primera_deteccion': b['temporal_placa'].iloc[0],
                    'duracion_min': duracion.total_seconds() / 60
                })


# B) Frecuencia anómala de picos severos

df_peaks_all = (
    df_peaks_all
    .sort_values('temporal_placa')
    .drop_duplicates(subset=['maquina','fase','temporal_placa'])
)

for (maq, fase), gp in df_peaks_all.groupby(['maquina','fase']):

    gp = gp.sort_values('temporal_placa').set_index('temporal_placa')

    s = pd.Series(1, index=gp.index)
    cnt = s.rolling('60min').sum()

    viol = cnt[cnt > cant_min_picos]

    if not viol.empty:
        t_fin = viol.index[0]
        t_ini = t_fin - tiempo_min_picos

        alarmas.append({
            'maquina': maq,
            'tipo_alarma': 'FRECUENCIA_PICOS',
            'descripcion': 'Frecuencia anómala de picos severos de corriente (>10 en 60 min)',
            'fase': fase.upper(),
            'valor_medido':cnt.max(),
            'valor_limite': cant_min_picos,
            'fecha_primera_deteccion': t_ini,
            'duracion_min': tiempo_min_picos.total_seconds() / 60
        })

df_alarmas = pd.DataFrame(alarmas)
df_alarmas


,maquina,tipo_alarma,descripcion,fase,valor_medido,valor_limite,fecha_primera_deteccion,duracion_min
0,PLE1,SOBRECORRIENTE,Sobrecorriente sostenida por encima del límite,R,17.316908,17,2026-01-10 00:13:59-03:00,11.483333
1,PLE1,SOBRECORRIENTE,Sobrecorriente sostenida por encima del límite,S,18.660106,18,2026-01-09 18:19:13-03:00,19.600000
2,PLE1,SOBRECORRIENTE,Sobrecorriente sostenida por encima del límite,S,19.094125,18,2026-01-09 18:40:51-03:00,15.983333
3,PLE1,SOBRECORRIENTE,Sobrecorriente sostenida por encima del límite,S,18.423510,18,2026-01-09 23:58:55-03:00,26.550000
4,PLE1,SOBRECORRIENTE,Sobrecorriente sostenida por encima del límite,T,18.771630,17,2025-12-30 00:00:14-03:00,53.300000
5,PLE1,SOBRECORRIENTE,Sobrecorriente sostenida por encima del límite,T,17.262980,17,2025-12-30 05:08:32-03:00,5.500000
6,PLE1,SOBRECORRIENTE,Sobrecorriente sostenida por encima del límite,T,17.146687,17,2025-12-30 05:53:03-03:00,5.983333
7,PLE1,SOBRECORRIENTE,Sobrecorriente sostenida por encima del límite,T,17.354562,17,2025-12-30 11:59:31-03:00,22.016667
8,PLE1,SOBRECORRIENTE,Sobrecorriente sostenida por encima del límite,T,17.235594,17,2025-12-30 22:14:32-03:00,10.483333
9,PLE1,SOBRECORRIENTE,Sobrecorriente sostenida por encima del límite,T,17.730256,17,2026-01-09 18:28:43-03:00,10.100000
